# INSTALL: packages

In [17]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [18]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [19]:
import os
import pickle
from WalletClustering_MIH import iterMultiInputClustering_chunks
from graphviz import Digraph
import pandas as pd

# EXECUTE: WalletClustering_AddressIdentification notebook

In [20]:
%run WalletClustering_AddressIdentification.ipynb # also includes running WalletClustering_neo4jConnect notebook
# methods & variables of notebook can be referenced

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



['12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8', '1348ThkNoDupq1bws95diMiL8haGs61K7M', '13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc', '13Pcmh4dKJE8Aqrhq4ZZwmM1sbKFcMQEEV', '15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf', '15soXrE3NJBMkkQhrccXonTT9bpjpPvE67', '164fawNZVwsR5SamAJypvCMtkMx4Xv1B3f', '179bzhS4FY7qLDza9YjuorhWyXVVYZu2YH', '17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD', '17UUXDzPGkMwWrabhtk7YCha88tSoua2Vr', '19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW', '19XVEDZCGVMA9WCF1qUayxtnjUnyD7zDDQ', '1A7pDH1EdrkH9YZtsPnc8uzirBFnAN9Eay', '1BPf9qr7M5xUgNHUYtrQtEKvUKcyERzXao', '1C6hetVWVXZnS6P2BYBNu5Y1ZJ57JyXGac', '1DrhHEkv42JVwiDQNi28JFdSuiSGgPNXwP', '1EDcKCRypUTFoTZbxDWF9MBAT4W7XUGB32', '1EfmRn6Bp3cjrTBubaH8MzRRc2ikSjNGXw', '1EnX6BuJiGWydqXJT9BN5dSvfLg3QW4Mdz', '1EVTZmTMqZPMzGxsug9TXBtvPJZH8dXSCK', '1EYya5dfNvuYDwpeboGKBtkXzJcEHMCQXR', '1GALPyvUDDXqA6H2eHQ9Y1yidfQ6T1Drvn', '1GC2SjzCyCwxo1uxTi28oqn9L3mJj7bLPs', '1Gg25VzQkqCizXHNSNet4RoysLEe19su4s', '1JpSBaUwrZaEgmsYka7mzm9t3Z4syyaw7A', '1LhRW1msre1cFgT7fBY2BRrZ4ANMPwVj9u', '1Lm9BCDUK

You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


                                     address              association
1         1CrhRvGShwmQkrGcqFzDDwDDBiEhKpHAqF               anxpro.com
2         1AbH5pFqocUYVhRqPAtJruVr5DMAxEANmn               anxpro.com
4         1FM8vDXinj2N5LoTXX5J4MSGTCn2pPkqRm               anxpro.com
7         18g2RSp4MxzkyNkigpYDejP2jMgkJ2Po9A               anxpro.com
18        195rycdSmRf1tQDztryRkZWKaFgFwM1PPG              bittrex.com
...                                      ...                      ...
21606795  1JiTqsNvfjkGSYVHMKHpWbNwEfN2LUhjKM  unknown_large_wallet_03
21606796  1HoNkBpNPkcznGWe6nUeLqffeLQN1TwhFJ  unknown_large_wallet_03
21606797  1BmvAcQPLcFve9KpNigfF591PyQnZwNDyQ  unknown_large_wallet_03
21606798  1GNw9S6hSzVQ83gUkP36ypjyrH3d6YZRm7  unknown_large_wallet_03
21606799  1McAmDN95SGxaTMmymaN8QzzTGJADM3aZy  unknown_large_wallet_03

[13983352 rows x 2 columns]
['hashnest.com', 'korbit.co.kr', 'bitcurex.com', 'satoshiroulette.com', 'bitcoinpokertables.com', 'bitcoin.de', 'betcoin.ag', 'btcc

In [21]:
# only to remember lists
assocBlacklist = pickle.load(open('output\\assocBlacklist.pickle', 'rb'))

# DEFINE: Make graph of flows

## check for blacklisted association

In [22]:
#check if association is part of blacklisted associations
def assocBlacklistCheck(association, blacklistToCheckAgainst = assocBlacklist):
    if association in assocBlacklist:
        return True
    else:
        return False

## query for input address association

In [23]:
def getAssoc(address, blacklistCheck = 'on'):
    # return association of input address
    assocTemplate = '''
    MATCH (a:Address {address: "%s"})
    RETURN a.association
    '''

    association = conn.query(assocTemplate % address, db='neo4j')
    
    if association[0][0] is None:
        # run MI heuristic on address and query again
        addresses, association = iterMultiInputClustering_chunks(address) #should also flag new association in neo4J
        return association

    else:
        if blacklistCheck == 'on':
            assocBlacklist = createAssocBlacklist()
            if assocBlacklistCheck(association, assocBlacklist):
                #throw exception in case association is blacklisted
                raise Exception('Blacklist')
            else:
                return association[0][0]
        else:
            return association[0][0]

## gather input and output addresses to association

In [24]:
def get_Inputs(query_Input):
    response = conn.query(query_Input, db='neo4j')
    rows_list = []
    for record in response:
        dict1 = {}
        new_row = {'Input_Addresses':record[0]._properties["address"],
        'Value_Received_from_Inp':record[1], 'Value_Send_from_Inp':record[2], 
        'Transaction_ID':record[3], 'Association':record[4]}
        dict1.update(new_row)
        rows_list.append(dict1)
    input_df = pd.DataFrame(rows_list) 
    return input_df  

In [25]:
def get_Outputs(query_Output):
    response = conn.query(query_Output, db='neo4j')
    rows_list = []
    for record in response:
        dict1 = {}
        new_row = {'Output_Addresses':record[0]._properties["address"],
        'Value_send_to_Outp':record[1], 'Value_rec_from_Outp':record[2],
        'Transaction_ID':record[3], 'Association':record[4]}
        dict1.update(new_row)
        rows_list.append(dict1)
    output_df = pd.DataFrame(rows_list) 
    return output_df

In [26]:
#get all addresses and their associations that are either output or input to an association (user)
def getInputs_Outputs(association):
    
    query_for_Outputs = """
    Match (a:Address {association: "%s"})-[send:SENDS]->(t:Transaction)-[r:RECEIVES]->(out:Address)
    return out as Output_Addresses, send.value as Value_send_to_Outp,r.value as Value_rec_from_Outp,
     t.txid as Transaction_ID, out.association as Association"""

    userOutput = get_Outputs(query_for_Outputs % association)

    query_for_Inputs = """
    Match (input:Address)-[s:SENDS]->(t:Transaction)-[r:RECEIVES]->(a:Address {association: "%s"})
    return input as Input_Addresses, r.value as Value_Received_from_Inp,s.value as Value_Send_from_Inp,
    t.txid as Transaction_ID, input.association as Association"""

    
    userInput = get_Inputs(query_for_Inputs % association)

    return userOutput, userInput

## query for input and output address asociations

In [27]:
def buildFlowGraph(inputAddress):
    try:
        association = getAssoc(inputAddress)
    except:
        raise Exception('Association of queried address is blacklisted.') # adjust to use case
    print('Queried address: '+inputAddress)
    print('Association of queried address: '+association)
    
    #export centerAssoc to file
    with open('output\\flowGraph_Center.pickle', 'wb') as export:
        pickle.dump(association, export)

    outputs, inputs = getInputs_Outputs(association)
    #print(outputs)
   
    outputAssocs = []
    for index, o in outputs.iterrows():
        if o['Association'] is not None:
            outputAssocs.append(o['Association'])
        else:
           outputAssocs.append(getAssoc(o['Output_Addresses'], 'Blacklist_allowed')) #allows blacklisted associations to be shown as receiving nodes. 
    outputAssocs = list(set(outputAssocs))
    
    #export outputAssoc to file
    with open('output\\flowGraph_Output.pickle', 'wb') as export:
        pickle.dump(outputAssocs, export)
    
    print('Receiving associations of outgoing transactions:')   
    print(outputAssocs)
    
    inputAssocs = []
    for index, i in inputs.iterrows():
        if i['Association'] is not None:
            inputAssocs.append(i['Association'])
        else:
            inputAssocs.append(getAssoc(i['Input_Addresses'], 'Blacklist_allowed')) #allows blacklisted associations to be shown as sending nodes.
    inputAssocs = list(set(inputAssocs))
    
    #export inputAssoc to file
    with open('output\\flowGraph_Input.pickle', 'wb') as export:
        pickle.dump(inputAssocs, export)
    
    print('Sending associations of incoming transactions:')
    print(inputAssocs)

    # build graph (adjust to use case)
    dot = Digraph()
    dot.node(association, association)
        
    for outNode in outputAssocs:
        dot.node(outNode, outNode)
        dot.edge(association, outNode)

    for inNode in inputAssocs:
        dot.node(inNode, inNode)
        dot.edge(inNode, association)
        
    return dot

## include transaction information in graph

In [28]:
def getUserTransactionData(association, outputNodes, inputNodes):
    userOutputTransactionDataTemplate = '''
        MATCH (u:Address{association:"%s"})-[:SENDS]->(t:Transaction)-[:SENDS]->(o:Address{association:"%s"}),
        (u:Address)-[:SENDS]->(t:Transaction)-[:SENDS]->(o:Address)
        RETURN DISTINCT o
        ''' 

# RUN: Make graph of flows

In [29]:
inputAddress = '12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8'

In [32]:
graph = buildFlowGraph(inputAddress)
graph.render(view=True)

Queried address: 12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8
Association of queried address: yYiOdGN2a2TGR8GBwrMtk1CkMAFUcaan
Receiving associations of outgoing transactions:
['yYiOdGN2a2TGR8GBwrMtk1CkMAFUcaan', 'ze3ws3ckYylbZwpXj6xNRXSBeRkenIzY']
Sending associations of incoming transactions:
['6LjxlCvzxnVaAMo3QuvCyAuyG3fyCyHD', 'FMKb71RPmguLvGGijceGfTeZJWIyYdFI', 'yYiOdGN2a2TGR8GBwrMtk1CkMAFUcaan', 'ze3ws3ckYylbZwpXj6xNRXSBeRkenIzY']


'Digraph.gv.pdf'